In [1]:
from pyspark.ml.feature import VectorAssembler
from synapse.ml.isolationforest import IsolationForest

StatementMeta(cybersparkpool, 30, 2, Finished, Available, Finished)

In [2]:
# Load feature table
feature_df = spark.read.table("model_features").fillna(0)

# Assemble features
feature_columns = [col for col in feature_df.columns if col != 'user']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_df = assembler.transform(feature_df).select("user", "features")

# Train Isolation Forest
iforest = IsolationForest(
    numEstimators=100, # number of trees
    contamination=0.07, # assumes 7% of data is "anomalous"
    featuresCol="features",
    predictionCol="prediction" )

model = iforest.fit(assembled_df)
scored_df = model.transform(assembled_df)

StatementMeta(cybersparkpool, 30, 3, Finished, Available, Finished)

In [3]:
# Save results
scored_df.write.mode("overwrite").saveAsTable("user_anomaly_scores")

StatementMeta(cybersparkpool, 30, 4, Finished, Available, Finished)